In [2]:
import pandas as pd 
import numpy as np
import nlp.wordbags as wb 
from nlp.word2vec import Word2WordPrediction
from nlp.w2w_readers import WordEmbeddings
import pickle


## Esempio minimo di Word2Vec sia come predittore che come embedding 

In [48]:
corpus = [
    ['montagna', 'neve', 'ghiaccio', 'vetta', 'bosco', 'freddo'],
    ['spiaggia', 'sabbia', 'mare', 'onda', 'sole', 'caldo'],
    ['deserto', 'duna', 'cactus', 'sabbia', 'sole', 'arido'],
    ['foresta', 'tropicale', 'liana', 'sole', 'umido', 'caldo'],
    ['lago', 'abete', 'vento', 'neve', 'bosco', 'freddo'],
    ['ghiaccio', 'artico', 'tormenta', 'neve', 'iceberg', 'gelido'],
    ['duna', 'oasi', 'deserto', 'cactus', 'arido']
]

In [49]:
bow = wb.Bow(corpus=corpus)
data_loader, inputs, outputs = bow.one_hot_skip_gram_dataloader(window=4, batch=2)

In [50]:
for doc in corpus:
    print(" | ".join(doc))

montagna | neve | ghiaccio | vetta | bosco | freddo
spiaggia | sabbia | mare | onda | sole | caldo
deserto | duna | cactus | sabbia | sole | arido
foresta | tropicale | liana | sole | umido | caldo
lago | abete | vento | neve | bosco | freddo
ghiaccio | artico | tormenta | neve | iceberg | gelido
duna | oasi | deserto | cactus | arido


## Predittore semplice

### Initialize and training the model

In [51]:
w2w = Word2WordPrediction(input_dim=bow.size, hidden_dim=4)
epochs = 150
learning_rate = 0.001
history = w2w.train(data_loader=data_loader, epochs=epochs, 
                    learning_rate=learning_rate)
print(history[0], history[-1])

100%|██████████| 150/150 [00:00<00:00, 315.95it/s]

15.233318692161923 14.620825404212589


In [52]:
model = WordEmbeddings(words=bow, model=w2w)

### Predict

In [53]:
search = 'montagna'
y_pred = model.predict(search, topk=10)
y_pred

neve        0.936070
ghiaccio    0.016087
bosco       0.010349
freddo      0.005761
tormenta    0.003302
artico      0.003214
montagna    0.003113
iceberg     0.003099
vetta       0.002938
abete       0.002779
dtype: float32

### Pseudo generazione

In [54]:
start = "iceberg"
testo = [start]
for i in range(6):
    p = model.predict(start)
    next_word = np.random.choice(p.index.values, p=p.values / p.values.sum())
    testo.append(next_word)
    start = next_word

In [55]:
testo

['iceberg', 'neve', 'lago', 'neve', 'ghiaccio', 'neve', 'lago']

### Embeddings

In [56]:
model[search]
model['spiaggia']

array([ 0.82992315,  0.83166915, -0.947049  , -0.9722851 ], dtype=float32)

In [57]:
search = "ghiaccio"
y_sim = model.most_similar(search, topk=10)
y_sim

ghiaccio    1.000000
artico      0.998472
vetta       0.998028
gelido      0.996586
abete       0.996195
tormenta    0.992974
vento       0.988713
iceberg     0.987923
bosco       0.984885
montagna    0.982684
Name: ghiaccio, dtype: float32

## Compositionality

In [63]:
answer, a_vector = model.analogy('deserto', 'duna', 'mare')
print(answer)

sabbia


In [59]:
model.common_meanings(words=['bosco', 'foresta', 'montagna', 'freddo'])

vetta       0.967534
bosco       0.966688
iceberg     0.966674
artico      0.961259
montagna    0.959348
ghiaccio    0.958091
abete       0.952729
gelido      0.952454
freddo      0.919057
tormenta    0.918570
dtype: float32

In [60]:
model.spot_odd_one(words=['montagna', 'vetta', 'ghiaccio', 'sabbia'])

sabbia     -0.698245
montagna    0.942648
vetta       0.968044
ghiaccio    0.975802
dtype: float32

In [62]:
model.search(positive=['deserto', 'mare'], negative=['duna'])

sabbia       0.967731
caldo        0.961940
onda         0.953320
spiaggia     0.911282
mare         0.884079
deserto      0.519593
arido        0.427811
sole         0.411034
liana        0.239419
tropicale    0.217112
dtype: float32

In [46]:
model.search(positive=['mare'], negative=['caldo'])

abete      0.720306
vento      0.690060
lago       0.537278
neve       0.484236
cactus     0.417193
duna       0.410956
arido      0.394200
sole       0.287713
deserto    0.260793
freddo     0.075906
dtype: float32